In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
def read_output(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)

    results = pd.DataFrame(data.get("Payload")['body'])
    cols = ["execution_time", "cost", "glue_job_name", "use_case", "proportion", "scale"]
    results = results[cols]
    results.rename({"glue_job_name": "compute"}, axis=1, inplace=True)
    results["execution_time_min"] = results["execution_time"] / 60
    results = results.groupby(["compute", "use_case", "proportion"])[["execution_time_min", "cost"]].mean().reset_index()
    return results

In [83]:
path_100 = '../../../iceberg-evaluation/src/data_curation/analysis/output/output_100_scale.json'
path_3000 = '../../../iceberg-evaluation/src/data_curation/analysis/output/output_3000_scale.json' 

scale_100 = read_output(output_file=path_100)
scale_3000 = read_output(output_file=path_3000)
scale_100['scale'] = '0.1TB'
scale_3000['scale'] = '3TB'
costs = pd.concat([scale_100, scale_3000], ignore_index=False)
costs['compute'] = costs.compute.str.replace('_iceberg' , '')
costs

,compute,use_case,proportion,execution_time_min,cost,scale
0,athena,bulk_insert,0.001,0.782642,0.102804,0.1TB
1,athena,bulk_insert,0.01,0.765433,0.102804,0.1TB
2,athena,bulk_insert,0.1,0.780183,0.102804,0.1TB
3,athena,bulk_insert,0.99,0.788317,0.102804,0.1TB
4,athena,scd2_complex,0.001,0.507433,0.146249,0.1TB
5,athena,scd2_complex,0.01,0.934750,0.147930,0.1TB
6,athena,scd2_complex,0.1,4.502750,0.163762,0.1TB
7,athena,scd2_simple,0.001,0.466500,0.146291,0.1TB
8,athena,scd2_simple,0.01,0.646333,0.148338,0.1TB
9,athena,scd2_simple,0.1,1.811450,0.168040,0.1TB


In [84]:
bulk_insert = costs.loc[costs['use_case']=='bulk_insert']
bulk_insert = bulk_insert.groupby(by=['scale','compute']).mean('numeric_only=True').reset_index()
bulk_insert

,scale,compute,execution_time_min,cost
0,0.1TB,athena,0.779144,0.102804
1,0.1TB,glue,3.552083,2.184111
2,3TB,athena,16.544658,2.147902
3,3TB,glue,9.329167,5.734346


In [85]:
def plot_bar_chart(df, title):

# Define new colors for compute types
    compute_colors = {"athena": "#1971c2", "glue": "#e64980"}

    # Create the subplots with separate Y-axes
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Query Duration", "Query Cost"))

    # Add Execution Time bars
    for compute in df['compute'].unique():
        compute_df = df[df['compute'] == compute]
        fig.add_trace(go.Bar(
            x=compute_df['scale'],
            y=compute_df['execution_time_min'],
            name=f'{compute} Execution Time',
            marker_color=compute_colors[compute],
            showlegend=False
        ), row=1, col=1)

    # Add Cost bars
    for compute in df['compute'].unique():
        compute_df = df[df['compute'] == compute]
        fig.add_trace(go.Bar(
            x=compute_df['scale'],
            y=compute_df['cost'],
            name=f'{compute}',
            marker_color=compute_colors[compute],
            showlegend=True
        ), row=2, col=1)

    # Update layout to have different Y-axis titles and formatting
    fig.update_layout(
        title=title,
        barmode='group',
        height=700,
        width=600,
        # Remove background color
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper

        # Increase the font size of the legend
        legend=dict(
            font=dict(size=30)  # Increase legend font size
        ),

    )
    
    # Increase the size of the subplot titles
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=30)  
    
    # Update the X-axis to increase font size
    fig.update_xaxes(tickfont=dict(size=30))  
    
    # Update Y-axes to hide them completely
    fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=True, zerolinecolor='grey')

    # Display the plot
    fig.show()


In [86]:
plot_bar_chart(bulk_insert, None)

In [87]:
scd2 = costs.loc[(costs['use_case']=='scd2_complex') & (costs['proportion']=='0.001')]
scd2

,compute,use_case,proportion,execution_time_min,cost,scale
4,athena,scd2_complex,0.001,0.507433,0.146249,0.1TB
14,glue,scd2_complex,0.001,6.000000,3.604333,0.1TB
4,athena,scd2_complex,0.001,7.052833,3.252361,3TB


In [88]:
plot_bar_chart(scd2, None)

In [5]:
costs = pd.read_csv('costs.csv')

In [7]:
# Convert the 'Date' column to datetime
costs['Date'] = pd.to_datetime(costs['Date'])

# Create a bar chart using Plotly
fig = go.Figure()

# Adding the bars for Glue costs
fig.add_trace(go.Bar(
    x=costs['Date'],
    y=costs['Glue'],
    name='Glue',
    marker_color='#e64980'
))

# Adding the bars for Athena costs
fig.add_trace(go.Bar(
    x=costs['Date'],
    y=costs['Athena'],
    name='Athena',
    marker_color="#1971c2"
))

# Update the layout for the chart
fig.update_layout(
    title=dict(text='Monthly Service Costs', x=0.5, y=0.85, font=dict(size=20)),
    barmode='group',
    height=500,
    width=500,
    # Remove background color
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper
    legend=dict(font=dict(size=20), x=0.8, y=0.95)
)

fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=True, zerolinecolor='grey')

fig.update_xaxes(tickfont=dict(size=20)) 

# Display the figure
fig.show()